In [1]:
import os
import pandas as pd
import yaml

In [2]:
DATASET_PARAMS = yaml.load(open("configs/dataset.yaml"), Loader=yaml.SafeLoader)['DATASET']
PREPROCESSING_PARAMS = yaml.load(open("configs/preprocessing.yaml"), Loader=yaml.SafeLoader)['PREPROCESSING']
EFFICIENTCAPSNET_PARAMS = yaml.load(open("configs/efficientcapsnet.yaml"), Loader=yaml.SafeLoader)['CAPSNET']

In [3]:
categories =  DATASET_PARAMS['CATEGORIES']

BASE_DIR = os.getcwd()
DATASET_DIR = os.path.join(BASE_DIR, DATASET_PARAMS['DATA_PATH'])

TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
TEST_DIR = os.path.join(DATASET_DIR, "test")

In [4]:
def get_filenames(dataset_path):
    # Get the list of file names
    filenames = os.listdir(dataset_path)
    filenames.sort()

    return filenames

def generate_filenames_df(dataset_dir, categories):
    full_filenames = {"image_path": [], "class": []}

    for category in categories:
        full_filenames_temp = get_filenames(os.path.join(dataset_dir, category))

        for i in range(len(full_filenames_temp)):
            full_filenames_temp[i] = os.path.join(dataset_dir, category, full_filenames_temp[i])

        full_filenames["image_path"].extend(full_filenames_temp)
        full_filenames["class"].extend([category] * len(full_filenames_temp))

    full_filenames_df = pd.DataFrame.from_dict(full_filenames)

    return full_filenames_df

In [5]:
train_filenames_df = generate_filenames_df(TRAIN_DIR, categories)
val_filenames_df = generate_filenames_df(VAL_DIR, categories)

## Pre Processor

In [6]:
from src.preprocessing import preprocess

train_transform = preprocess(
    rotation_range=PREPROCESSING_PARAMS['ROTATION_RANGE'],
    width_shift_range=PREPROCESSING_PARAMS['WIDTH_SHIFT_RANGE'],
    height_shift_range=PREPROCESSING_PARAMS['HEIGHT_SHIFT_RANGE'],
    brightness_range=PREPROCESSING_PARAMS['BRIGHTNESS_RANGE'],
    zoom_range=PREPROCESSING_PARAMS['ZOOM_RANGE'],
    horizontal_flip=PREPROCESSING_PARAMS['HORIZONTAL_FLIP'],
    vertical_flip=PREPROCESSING_PARAMS['VERTICAL_FLIP'],
    channel_shift_range=PREPROCESSING_PARAMS['CHANNEL_SHIFT_RANGE'],
    fill_mode=PREPROCESSING_PARAMS['FILL_MODE'],
    drop_out=PREPROCESSING_PARAMS['DROP_OUT'],
    shear_range=PREPROCESSING_PARAMS['SHEAR_RANGE']
    )

val_transform = preprocess() # only rescaling


TypeError: preprocess() missing 1 required positional argument: 'target_input_size'

## Train loader

In [ ]:
from src.loader import Loader
train_loader = Loader(train_filenames_df, 
                     batch_size=EFFICIENTCAPSNET_PARAMS['BATCH_SIZE'], 
                     num_workers=EFFICIENTCAPSNET_PARAMS['NUM_WORKERS'], 
                     transform=train_transform, 
                     shuffle=True)

val_loader = Loader(val_filenames_df, 
                    batch_size=EFFICIENTCAPSNET_PARAMS['BATCH_SIZE'], 
                    num_workers=EFFICIENTCAPSNET_PARAMS['NUM_WORKERS'], 
                    transform=val_transform, 
                    shuffle=False)


# Model

In [8]:
from src.model import EfficientCapsNet

model = EfficientCapsNet(input_size=(EFFICIENTCAPSNET_PARAMS['INPUT_SIZE']))

# Loss
from src.loss import MarginLoss

margin_loss = MarginLoss()

# Optimizer
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=EFFICIENTCAPSNET_PARAMS['LEARNING_RATE'])

# Train
from src.train import train
import torch

train(model=model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    criterion=margin_loss, 
    optimizer=optimizer, 
    num_epochs=EFFICIENTCAPSNET_PARAMS['NUM_EPOCHS'], 
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Training...


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:04<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [3, 548, 635] at entry 0 and [3, 547, 635] at entry 1